In [0]:
#@title Imports

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import queue
import pydot
import json
from networkx.drawing.nx_pydot import graphviz_layout
from collections import deque, OrderedDict
from google.colab import files
from __future__ import generators

In [0]:
class PairTransitionsAlgorithm(object):

    def __init__(self, switches, weights):
        self._switches = switches
        self._weights = weights
        self.tree_edges = set()
        self.replacement_edges = set()
        self.current_route = Route([])
        self.previous = {}

    @property
    def tree(self):
        return self.tree_edges

    def compute_shortest_route(self, src, dst):
        dijkstra = DijkstraAlgorithm(
            self._weights, self._switches)
        self.current_route = \
            dijkstra.compute_shortest_path(src, dst)
        self.previous = dijkstra.previous
        return self.current_route

    def recompute_shortest_route(self):
        if len(self.current_route) > 1:
            return self.compute_shortest_route(
                self.current_route.source,
                self.current_route.destination)
        return self.current_route

    def count_pair_transitions(self):
        old_tree_edges = self.tree_edges
        self.collect_statistics()
        return len(self.tree_edges - old_tree_edges) / 2

    def collect_statistics(self):
        all_edges = self.unpack_edges_from_weights(
            self._weights)
        self.tree_edges = self.unpack_edges_from_previous(
            self.previous)
        self.replacement_edges = \
            all_edges - self.tree_edges

    def unpack_edges_from_weights(self, weights):
        all_edges = set()
        for v1 in weights.keys():
            for v2 in weights[v1].keys():
                all_edges.add((v1, v2))
        return all_edges

    def unpack_edges_from_previous(self, previous):
        tree_edges = set()
        for item in previous:
            if previous[item]:
                tree_edges.add((item, previous[item]))
                tree_edges.add((previous[item], item))
        return tree_edges

In [0]:
#@title Drawing

def draw_graph(g, node_colors, edge_colors={}, distances={}, caption='', count=0):
  plt.figure(figsize=(10,6))
  edge_labels = nx.get_edge_attributes(g, 'weight')
  pos = graphviz_layout(g)
  nx.draw(
      g, 
      pos=pos, 
      node_color=node_colors.values(), 
      with_labels=True, 
      font_color='white')
  nx.draw_networkx_edges(
      g, 
      pos=pos, 
      edgelist=edge_colors.keys(), 
      width=5, 
      alpha=0.5, 
      edge_color=edge_colors.values())
  nx.draw_networkx_edge_labels(
      g, 
      pos, 
      edge_labels=edge_labels)
    
  for node, position in zip(g.nodes(), pos.values()):
    if node in distances.keys():
      plt.text(
          position[0], 
          position[1]+10, 
          s=f'{distances[node]}', 
          bbox=dict(facecolor='red', alpha=0.5), 
          horizontalalignment='center')
  plt.title(caption, loc='left')
  #plt.savefig(f'graph{count}.png', edgecolor='k', bbox_inches="tight")
  #files.download(f'graph{count}.png')
  plt.show()

def draw_shortest_paths(paths, suptitle, count=0):
  fig = plt.figure(1, figsize=(10,6))
  fig, ax = plt.subplots(1, len(paths), num=1)
  for path, i in zip(paths, range(0, len(paths))):
    plt.sca(ax[i] if len(paths)>1 else ax)
    graph = nx.Graph()
    graph.add_path(path['path'])
    pos = graphviz_layout(graph)
    nx.draw(
      graph, 
      pos, 
      ax=(ax[i] if len(paths)>1 else ax),
      with_labels=True, 
      font_color='white')

    (ax[i] if len(paths)>1 else ax).set_title(f'Path {i+1} : Path cost: {path["cost"]}')

  plt.suptitle(suptitle)
  #plt.savefig(f'list{count}.png', edgecolor='k', bbox_inches="tight")
  #files.download(f'list{count}.png')
  plt.show()

In [0]:
#@title Dijkstra & Path extraction

## Computes the shortest path from a source to a sink in the supplied graph.
#
# graph A digraph of class Graph.
# initial The source node of the graph.
# end The sink node of the graph.
#
# @retval {} Dictionary of path and cost or if the node_end is not specified,
# the distances and previous lists are returned.
#
def dijkstra(graph, initial, end=None):
  distances = {initial: 0}
  shortest_tree = {}
  nodes = list(graph.nodes())

  while nodes: 
    min_node = None
    for node in nodes:
      if node in distances:
        if min_node is None:
          min_node = node
        elif distances[node] < distances[min_node]:
          min_node = node

    if min_node is None:
      break

    nodes.remove(min_node)
    current_weight = distances[min_node]
    
    for neighbor in graph[min_node]:
      weight = current_weight + graph[min_node][neighbor]['weight']
      if neighbor not in distances or weight < distances[neighbor]:
        distances[neighbor] = weight
        shortest_tree[neighbor] = min_node

  if end:
    return ({'cost': distances[end],
            'path': path(shortest_tree, initial, end)}, distances, shortest_tree)
  else:
    shortest_tree = [(shortest_tree[node], node) for node in list(shortest_tree.keys())] 
    return (distances, shortest_tree) 

## Finds a paths from a source to a sink using a supplied previous node list.
#
# previous A list of node predecessors.
# node_start The source node of the graph.
# node_end The sink node of the graph.
#
# @retval [] Array of nodes if a path is found, an empty list if no path is 
# found from the source to sink.
#
def path(previous, node_start, node_end):
  route = []

  node_curr = node_end    
  while True:
    route.append(node_curr)
    if previous[node_curr] == node_start:
      route.append(node_start)
      break
    elif not node_curr in previous:
      return []
    
    node_curr = previous[node_curr]
  
  route.reverse()
  return route

def path_length(graph, path):
  length = 0
  for edge in list(path):
    length += graph[edge[0]][edge[1]]['weight']
  return length

def path_as_edge_list(path):
  return [(path[i], path[i+1]) for i in range(0, path.length-1)]

# simple_paths = [(path_length1, path1), (path_length2, path2), ...]
# edges = [edge1, edge2, ...]
def shortest_path_without_edges(simple_paths, edges):
  for length, path in sorted(simple_paths, key=lambda x: x[0]):
    if len(set(edges).intersection(set(path_as_edge_list(path)))) == 0:
      return (length, path)
  else:
    raise Error('reee')

In [0]:
def pair_perm(graph, start):
  # paths from each node to start node with their lengths
  simple_paths = {}
  # how many incidental edges to each node form simple paths to start node
  routing_degree = {}
  # transition pairs
  transition_pairs = {}

  for node in list(graph.nodes()):
    simple_paths[node] = sorted([(path_length(path), path) for path in list(nx.all_simple_paths(graph, node, start))], key=lambda x: x[0])
    routing_degree[node] = len(set([(x[0], x[1]) for x in simple_paths[node]])) 
  
  shortest_distances, shortest_tree_edges = dijkstra(graph, start)
  transition_edges = set(graph.edges()) - set(shortest_tree_edges)

  shortest_tree_inclusion_point = {}
  transition_inclusion_point = {}
  for edge in list(graph.edges()):
    second_shortest_path_without_edge = shortest_path_without_edges(simple_paths, [edge])
    second_shortest_path_ending_edge = (second_shortest_path_without_edge[1][-2], second_shortest_path_without_edge[1][-1])
    
    shortest_tree_inclusion_point[edge] = graph[edge[0]][edge[1]]['weight'] 
      + (second_shortest_path_without_edge[0] - shortest_distances[edge[1]])

    if routing_degree[edge[0]] > 2 and routing_degree[edge[1]] > 2:
      third_shortest_path_without_previous_edges = shortest_path_without_edges(simple_paths, [edge, second_shortest_path_ending_edge])
      third_shortest_path_ending_edge = (third_shortest_path_without_previous_edges[1][-2], third_shortest_path_without_previous_edges[1][-1])
      transition_inclusion_point[second_shortest_path_ending_edge] = graph[second_shortest_path_ending_edge[0]][second_shortest_path_ending_edge[1]]['weight'] 
        + (third_shortest_path_without_previous_edges[0] - shortest_distances[edge[1]])

  return 'what'

In [0]:
#@title Input data
# (u, v, weight)
graph = [
        (1, 2, 7),
        (1, 3, 9),
        (1, 6, 14),
        (2, 3, 10),
        (2, 4, 15),
        (3, 4, 11),
        (3, 6, 2),
        (4, 5, 6),
        (5, 6, 9)
        ]

G = nx.Graph()
G.add_weighted_edges_from(graph)

In [0]:
k = 3
start = 1
end = 6

In [0]:
list(nx.all_simple_paths(G, 1, 2))

[[1, 2],
 [1, 3, 2],
 [1, 3, 4, 2],
 [1, 3, 6, 5, 4, 2],
 [1, 6, 3, 2],
 [1, 6, 3, 4, 2],
 [1, 6, 5, 4, 2],
 [1, 6, 5, 4, 3, 2]]